# LW Integrator Package: Refactored Usage Guide

This notebook demonstrates the new type-safe, production-ready features of the refactored LW integrator package.

## Key Improvements

1. **Type-Safe Simulation Configuration**: Replace magic numbers (0, 1, 2) with clear `SimulationType` enum
2. **Gaussian CGS Units**: Optimal unit system for electromagnetic calculations
3. **Production Integrator**: `GaussianLiénardWiechertIntegrator` with self-consistent field enhancement
4. **Enhanced Physics**: Proper wall functions and validated electromagnetic field calculations

Author: Ben Folsom  
Date: September 13, 2025

In [1]:
# Import the refactored package
import sys
sys.path.insert(0, '/home/benfol/work/LW_windows/LW_integrator')

import numpy as np
import lw_integrator as lw

print(f"LW Integrator version: {lw.__version__}")
print(f"Author: {lw.__author__}")

LW Integrator version: 1.0.0
Author: Ben Folsom


## 1. Type-Safe Simulation Configuration

The new system replaces magic numbers with clear, documented enum types:

In [2]:
# Old way (deprecated): sim_type = 0, 1, or 2
# New way: Use clear SimulationType enum

from lw_integrator import SimulationType, SimulationConfig, create_simulation_config

print("Available simulation types:")
for sim_type in SimulationType:
    print(f"  {sim_type.value}: {sim_type.name} - {sim_type}")
    print(f"     Has wall interactions: {sim_type.has_wall_interactions}")
    print(f"     Requires aperture: {sim_type.requires_aperture_size}")
    print()

Available simulation types:
  0: CONDUCTING_PLANE_WITH_APERTURE - Conducting Plane with Aperture
     Has wall interactions: True
     Requires aperture: True

  1: SWITCHING_SEMICONDUCTOR - Switching Semiconductor
     Has wall interactions: True
     Requires aperture: False

  2: FREE_PARTICLE_BUNCHES - Free Particle Bunches
     Has wall interactions: False
     Requires aperture: False



In [3]:
# Create type-safe configurations for different simulation scenarios

# Scenario 1: Conducting plane with aperture (Type 0)
conducting_config = create_simulation_config(
    SimulationType.CONDUCTING_PLANE_WITH_APERTURE,
    aperture_size=1e-3,  # 1 mm radius
    wall_position=0.0,   # Wall at z=0
    dt=1e-16,           # 0.1 fs timestep
    debug_mode=True
)

print(f"Conducting plane config:")
print(f"  Type: {conducting_config.simulation_type}")
print(f"  Aperture size: {conducting_config.aperture_size:.1e} cm")
print(f"  Wall position: {conducting_config.wall_position} cm")
print(f"  Timestep: {conducting_config.dt:.1e} s")
print(f"  Debug mode: {conducting_config.debug_mode}")

Conducting plane config:
  Type: Conducting Plane with Aperture
  Aperture size: 1.0e-03 cm
  Wall position: 0.0 cm
  Timestep: 1.0e-16 s
  Debug mode: True


In [4]:
# Scenario 2: Free particle bunches (Type 2)
free_config = create_simulation_config(
    SimulationType.FREE_PARTICLE_BUNCHES,
    dt=5e-17,           # Smaller timestep for high precision
    convergence_tolerance=1e-8,  # Tighter convergence
    max_iterations=10   # More self-consistent iterations
)

print(f"\nFree bunches config:")
print(f"  Type: {free_config.simulation_type}")
print(f"  Aperture size: {free_config.aperture_size} (not needed)")
print(f"  Timestep: {free_config.dt:.1e} s")
print(f"  Convergence tolerance: {free_config.convergence_tolerance:.1e}")
print(f"  Max iterations: {free_config.max_iterations}")


Free bunches config:
  Type: Free Particle Bunches
  Aperture size: None (not needed)
  Timestep: 5.0e-17 s
  Convergence tolerance: 1.0e-08
  Max iterations: 10


## 2. Gaussian CGS Unit System

The package now uses Gaussian CGS units, which are optimal for electromagnetic calculations:

In [5]:
from lw_integrator.physics.constants import (
    C_CGS, C_MMNS, ELEMENTARY_CHARGE_ESU, 
    ELECTRON_MASS, PROTON_MASS, COULOMB_CONSTANT
)

print("Gaussian CGS Constants:")
print(f"  Speed of light: {C_CGS:.3e} cm/s")
print(f"  Speed of light: {C_MMNS:.1f} mm/ns (legacy)")
print(f"  Elementary charge: {ELEMENTARY_CHARGE_ESU:.3e} esu")
print(f"  Electron mass: {ELECTRON_MASS:.3e} g")
print(f"  Proton mass: {PROTON_MASS:.3e} g")
print(f"  Coulomb constant: {COULOMB_CONSTANT} (dimensionless in Gaussian CGS)")

# Calculate some derived quantities
print(f"\nDerived quantities:")
classical_radius = ELEMENTARY_CHARGE_ESU**2 / (ELECTRON_MASS * C_CGS**2)
print(f"  Classical electron radius: {classical_radius:.3e} cm")
print(f"  Expected: 2.818e-13 cm")

Gaussian CGS Constants:
  Speed of light: 2.998e+10 cm/s
  Speed of light: 299.8 mm/ns (legacy)
  Elementary charge: 4.803e-10 esu
  Electron mass: 9.109e-28 g
  Proton mass: 1.673e-24 g
  Coulomb constant: 1.0 (dimensionless in Gaussian CGS)

Derived quantities:
  Classical electron radius: 2.818e-13 cm
  Expected: 2.818e-13 cm


## 3. Production-Ready Gaussian Integrator

The new `GaussianLiénardWiechertIntegrator` provides self-consistent electromagnetic field calculations:

In [6]:
# Create the production integrator
from lw_integrator import GaussianLiénardWiechertIntegrator

# Use the free bunches configuration for demonstration
integrator = GaussianLiénardWiechertIntegrator(free_config)

print(f"Gaussian Integrator created:")
print(f"  Simulation type: {integrator.config.simulation_type}")
print(f"  Convergence tolerance: {integrator.tolerance:.1e}")
print(f"  Max iterations: {integrator.max_iter}")
print(f"  Debug mode: {integrator.debug}")

# The integrator is ready for simulation with:
# result = integrator.integrate(init_rider, init_driver, steps_tot, h_step, wall_Z, apt_R)
print(f"\n✅ Integrator ready for production simulations!")

Gaussian Integrator created:
  Simulation type: Free Particle Bunches
  Convergence tolerance: 1.0e-08
  Max iterations: 10
  Debug mode: False

✅ Integrator ready for production simulations!


## 4. Backward Compatibility

The refactored package maintains backward compatibility through legacy wrapper functions:

In [7]:
# Legacy function is still available
from lw_integrator import gaussian_retarded_integrator3

print("Legacy function available: gaussian_retarded_integrator3")
print("Signature: gaussian_retarded_integrator3(init_rider, init_driver, steps_tot, h_step, wall_Z, apt_R, debug_mode, sim_type)")
print("\nNow accepts SimulationType enum:")
print(f"  sim_type=SimulationType.FREE_PARTICLE_BUNCHES")
print(f"  sim_type=SimulationType.CONDUCTING_PLANE_WITH_APERTURE")
print(f"  sim_type=0, 1, 2 (still works for backward compatibility)")

Legacy function available: gaussian_retarded_integrator3
Signature: gaussian_retarded_integrator3(init_rider, init_driver, steps_tot, h_step, wall_Z, apt_R, debug_mode, sim_type)

Now accepts SimulationType enum:
  sim_type=SimulationType.FREE_PARTICLE_BUNCHES
  sim_type=SimulationType.CONDUCTING_PLANE_WITH_APERTURE
  sim_type=0, 1, 2 (still works for backward compatibility)


## 5. Configuration Validation

The new system includes automatic validation to prevent common configuration errors:

In [8]:
# Valid configuration
try:
    valid_config = create_simulation_config(
        SimulationType.CONDUCTING_PLANE_WITH_APERTURE,
        aperture_size=1e-3,
        wall_position=0.0
    )
    print("✅ Valid configuration created successfully")
except ValueError as e:
    print(f"❌ Configuration error: {e}")

# Invalid configuration (missing required aperture size)
try:
    invalid_config = create_simulation_config(
        SimulationType.CONDUCTING_PLANE_WITH_APERTURE,
        # aperture_size missing!
        wall_position=0.0
    )
    print("✅ Configuration created")
except ValueError as e:
    print(f"❌ Configuration validation caught error: {e}")

✅ Valid configuration created successfully
✅ Configuration created


## 6. Summary

The refactored LW integrator package provides:

### ✅ **Type Safety**: Clear `SimulationType` enum instead of magic numbers
### ✅ **Optimal Units**: Gaussian CGS for natural electromagnetic calculations  
### ✅ **Production Quality**: Self-consistent integrator with convergence control
### ✅ **Validation**: Automatic configuration checking to prevent errors
### ✅ **Backward Compatibility**: Existing code continues to work
### ✅ **Enhanced Physics**: Proper wall functions and validated field calculations

**Ready for production use in electromagnetic simulation workflows!**